In [1]:
from ogb.graphproppred import PygGraphPropPredDataset
from utils.dataset import OGBDataset_aug
from torch_geometric.loader import DataLoader
import time
import torch
import numpy as np
from utils.augmentors import *

In [2]:
aug1 = Identity()
aug2 = RandomChoice([RWSampling(num_seeds=1000, walk_length=10),
                           NodeDropping(pn=0.1),
                           FeatureMasking(pf=0.1),
                           EdgeRemoving(pe=0.1)], 1)


dataset_1 = PygGraphPropPredDataset(name="ogbg-molhiv", root="./data/")
dataset_2 = OGBDataset_aug(name="HIV", root="./data/", aug="none")

In [9]:
loader_1 = DataLoader(dataset_1, batch_size=512, num_workers=4)
loader_2 = DataLoader(dataset_2, batch_size=512, num_workers=4)

In [10]:
for i, batch in enumerate(loader_1):
    x1, edge_index1, edge_weight1 = aug1(batch.x, batch.edge_index, batch.edge_attr)
    x2, edge_index2, edge_weight2 = aug2(batch.x, batch.edge_index, batch.edge_attr)
    break
    
    

/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be r

In [13]:
x1.shape, edge_index1.shape, edge_weight1.shape, batch.batch.shape

(torch.Size([10551, 9]),
 torch.Size([2, 22324]),
 torch.Size([22324, 3]),
 torch.Size([10551]))

In [14]:
x2.shape, edge_index2.shape, edge_weight2.shape, batch.batch.shape

(torch.Size([10551, 9]),
 torch.Size([2, 20085]),
 torch.Size([20085, 3]),
 torch.Size([10551]))

In [4]:
perm2 = torch.randperm(len(aug2))
idx2 = perm2[:1]
aug_select2 = Compose([aug2[id] for id in idx2])

In [15]:
dataset_1[0]

Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)

In [ ]:
t_0 = time.time()
for i, batch in enumerate(loader_1):
    data, data_aug = batch
    print(i)
time.time() - t_0

In [ ]:
data

In [ ]:
data_aug

In [ ]:
t_0 = time.time()
for batch in loader_1:
    data = batch
    pass
time.time() - t_0

In [ ]:
def drop_nodes(data):
    node_num, _ = data.x.size()
    _, edge_num = data.edge_index.size()
    drop_num = int(node_num * 0.2)

    idx_drop = np.random.choice(node_num, drop_num, replace=False)
    idx_nondrop = [n for n in range(node_num) if not n in idx_drop]
    idx_dict = {idx_nondrop[n]: n for n in list(range(node_num - drop_num))}

    edge_index = data.edge_index.numpy()

    adj = torch.zeros((node_num, node_num))
    adj[edge_index[0], edge_index[1]] = 1
    adj[idx_drop, :] = 0
    adj[:, idx_drop] = 0
    edge_index = torch.nonzero(adj).t()

    data.edge_index = edge_index
    return data

In [3]:
from utils.loader import get_split_loader

In [4]:
ogb_list = ["hiv", "bbbp", "clintox", "tox21", "sider"]
tu_list = ["mutag", "dd", "proteins", "enzymes", "collab", "imdb-binary", "reddit-binary"]

In [7]:
dataloader, train_loader, val_loader, test_loader = get_split_loader(name="HIV", root="./data", train_ratio=0.6, val_ratio=0.2, batch_size=512)

In [12]:
temp = next(iter(dataloader))
temp.x.shape[-1]
temp.edge_attr.shape[-1]

/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/home/cjz/miniconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be r

9

In [53]:
val_y = torch.load("val_y.pt")
test_y = torch.load("test_y.pt")
val_pred = torch.load("val_pred.pt")
test_pred = torch.load("test_pred.pt")

In [54]:
val_y.nonzero()

(array([  51,  111,  140,  185,  284,  293,  297,  319,  320,  348,  449,
         473,  486,  575,  954, 1333, 1413, 1614, 1802, 1850, 1885, 1919,
        2007, 2008, 2009, 2053, 2105, 2216, 2244, 2245, 2251, 2391, 2406,
        2456, 2457, 2458, 2459, 2486, 2505, 2508, 2627, 2662, 2670, 2673,
        2690, 2693, 2747, 2758, 2779, 2780, 2781, 2782, 2783, 2833, 2890,
        2909, 2960, 2985, 2986, 2988, 3004, 3028, 3061, 3084, 3169, 3170,
        3176, 3177, 3223, 3317, 3353, 3416, 3461, 3662, 3755, 3798, 3799,
        3812, 3867, 3873, 4027]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [57]:
val_pred[:,1:2].max()

-0.6528204

In [36]:
val_y = torch.load("save_data/val_y.pt").cpu().numpy()
test_y = torch.load("save_data/test_y.pt").cpu().numpy()
val_pred = torch.load("save_data/val_pred.pt").cpu().numpy()
test_pred = torch.load("save_data/test_pred.pt").cpu().numpy()

In [37]:
val_y.nonzero()

(array([  51,  111,  140,  185,  284,  293,  297,  319,  320,  348,  449,
         473,  486,  575,  954, 1333, 1413, 1614, 1802, 1850, 1885, 1919,
        2007, 2008, 2009, 2053, 2105, 2216, 2244, 2245, 2251, 2391, 2406,
        2456, 2457, 2458, 2459, 2486, 2505, 2508, 2627, 2662, 2670, 2673,
        2690, 2693, 2747, 2758, 2779, 2780, 2781, 2782, 2783, 2833, 2890,
        2909, 2960, 2985, 2986, 2988, 3004, 3028, 3061, 3084, 3169, 3170,
        3176, 3177, 3223, 3317, 3353, 3416, 3461, 3662, 3755, 3798, 3799,
        3812, 3867, 3873, 4027]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [40]:
(val_pred > 0).sum()

14

In [43]:
from ogb.graphproppred import Evaluator
evaluator = Evaluator(name='ogbg-molhiv')
epoch_test_ROC = evaluator.eval({'y_pred': val_pred,
                                'y_true': val_y})['rocauc']
epoch_test_ROC

0.7549113266705859

In [50]:
torch.nn.functional.softmax(torch.from_numpy(val_pred))

/tmp/ipykernel_42539/777834420.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(torch.from_numpy(val_pred))


tensor([[0.8339, 0.1661],
        [0.8339, 0.1661],
        [0.8339, 0.1661],
        ...,
        [0.8339, 0.1661],
        [0.8339, 0.1661],
        [0.8339, 0.1661]])

In [58]:
val_pred.shape

(4113, 2)

In [60]:
val_y[:,0].sum()

81

In [64]:
torch.nn.functional.cross_entropy(torch.from_numpy(val_pred), torch.from_numpy(val_y[:,0]))

tensor(0.2134)

In [5]:
len(dataset_1.get_idx_split()['train'])

32901

In [ ]:
32901
